# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

# PG Diploma in Machine Learning and AI.

#### Developed by:
##### 1. Piyush Kadve - Group facilitator
##### 2. Ajith Rao

In [2]:
import numpy as np
import os
from matplotlib.pyplot import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 32

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
x = 30 # number of frames
y = 160 # image width
z = 160 # image height
channels=3
classes=5

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[x for x in range(0,x)]   #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size    # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =   10  # choose the number of epochs
print ('# epochs =', num_epochs)
num_batches = num_train_sequences//batch_size 
print(num_batches)

# training sequences = 663
# validation sequences = 100
# epochs = 10
20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

#### Experiment 1: Creating a model with 160x160 image size,epochs=10 and batch_size=32

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.layers import Dropout

#write your model here
model_1 = Sequential()

model_1.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(x,y,z,channels),
                 padding='same'))

model_1.add(Activation('relu'))
model_1.add(BatchNormalization())

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))

model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))

model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2,2,2)))


model_1.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))



#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(100, activation='relu'))
model_1.add(Dropout(0.5))


#softmax layer
model_1.add(Dense(classes, activation='softmax'))

2022-05-18 12:44:19.596676: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-05-18 12:44:19.596740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14803 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser = 'adam'
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 160, 160, 8)   656       
                                                                 
 activation (Activation)     (None, 30, 160, 160, 8)   0         
                                                                 
 batch_normalization (BatchN  (None, 30, 160, 160, 8)  32        
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 80, 80, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 80, 80, 16)    3472      
                                                                 
 activation_1 (Activation)   (None, 15, 80, 80, 16)    0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, cooldown=1, verbose=1)# write the REducelronplateau code here
earlystop = EarlyStopping( monitor="val_loss", min_delta=0,patience=10,verbose=1)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/10


2022-05-18 05:21:40.415274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


21/21 [==============================] - ETA: 0s - loss: 9.8836 - categorical_accuracy: 0.3439 Source path =  /home/datasets/Project_data/val ; batch size = 32
Batch:  4 Index: 32

Epoch 00001: saving model to model_init_2022-05-1805_21_29.335541/model-00001-9.88361-0.34389-1.65058-0.16000.h5
21/21 [==============================] - 194s 9s/step - loss: 9.8836 - categorical_accuracy: 0.3439 - val_loss: 1.6506 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 1.3998 - categorical_accuracy: 0.4058
Epoch 00002: saving model to model_init_2022-05-1805_21_29.335541/model-00002-1.39981-0.40580-1.98394-0.18750.h5
21/21 [==============================] - 125s 6s/step - loss: 1.3998 - categorical_accuracy: 0.4058 - val_loss: 1.9839 - val_categorical_accuracy: 0.1875 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 1.3358 - categorical_accuracy: 0.4286
Epoch 00003: saving model to model_init_2022-05

Not good results

#### Exp -2 :   Reduce the batch size to 20 and image dimensions to 60x60 with 30 frames 

In [13]:
channels=3
classes=5


x = 30 # number of frames
y = 60 # image width
z = 60 # image height
batch_size=20

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[x for x in range(0,x)]   #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size    # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


In [14]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =   10  # choose the number of epochs
print ('# epochs =', num_epochs)
num_batches = num_train_sequences//batch_size 
print(num_batches)

# training sequences = 663
# validation sequences = 100
# epochs = 10
33


In [15]:
#define model
model_b = Sequential()
model_b.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(x,y,z,channels), padding='same'))
model_b.add(Activation('relu'))
model_b.add(BatchNormalization())
model_b.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_b.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(BatchNormalization())
model_b.add(MaxPooling3D(pool_size=(2, 2, 2)))


model_b.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(BatchNormalization())
model_b.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_b.add(Flatten())
model_b.add(Dense(128, activation='relu'))
model_b.add(Dropout(0.25))
model_b.add(Dense(64, activation='relu'))
model_b.add(Dropout(0.25))
model_b.add(Dense(classes, activation='softmax'))

In [16]:
optimiser = 'adam'
model_b.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_b.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 60, 60, 16)    1312      
                                                                 
 activation_4 (Activation)   (None, 30, 60, 60, 16)    0         
                                                                 
 batch_normalization_4 (Batc  (None, 30, 60, 60, 16)   64        
 hNormalization)                                                 
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 15, 30, 30, 16)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 30, 30, 32)    13856     
                                                                 
 activation_5 (Activation)   (None, 15, 30, 30, 32)   

In [17]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [18]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, cooldown=1, verbose=1)# write the REducelronplateau code here
earlystop = EarlyStopping(monitor="val_loss", min_delta=0,patience=10,verbose=1)
callbacks_list = [checkpoint, LR,earlystop]

In [19]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [20]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 20
Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 2.2857 - categorical_accuracy: 0.2941Source path =  /home/datasets/Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2022-05-1805_39_49.403993/model-00001-2.28567-0.29412-2.05829-0.23000.h5
34/34 [==============================] - 190s 6s/step - loss: 2.2857 - categorical_accuracy: 0.2941 - val_loss: 2.0583 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 2.5819 - categorical_accuracy: 0.2353
Epoch 00002: saving model to model_init_2022-05-1805_39_49.403993/model-00002-2.58190-0.23529-2.79673-0.21000.h5
34/34 [==============================] - 62s 2s/step - loss: 2.5819 - categorical_accuracy: 0.2353 - val_loss: 2.7967 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 2.6365 - categorical_accuracy

#### Exp-3 :adding more layers and increase the no of epochs to 15

In [21]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

batch_size=20
num_epochs=15

In [22]:
model_b1 = Sequential()
model_b1.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(x,y,z,channels), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(x,y,z,channels), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_b1.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(MaxPooling3D(pool_size=(2, 2, 2)))


model_b1.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
model_b1.add(Activation('relu'))
model_b1.add(BatchNormalization())

model_b1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_b1.add(Flatten())
model_b1.add(Dense(128, activation='relu'))
model_b1.add(Dropout(0.25))
model_b1.add(Dense(64, activation='relu'))
model_b1.add(Dropout(0.25))
model_b1.add(Dense(classes, activation='softmax'))

In [23]:
optimiser = 'adam'
model_b1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_b1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_7 (Conv3D)           (None, 30, 60, 60, 16)    1312      
                                                                 
 activation_7 (Activation)   (None, 30, 60, 60, 16)    0         
                                                                 
 batch_normalization_7 (Batc  (None, 30, 60, 60, 16)   64        
 hNormalization)                                                 
                                                                 
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 16)    6928      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 16)    0         
                                                                 
 batch_normalization_8 (Batc  (None, 30, 60, 60, 16)   64        
 hNormalization)                                      

In [24]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [26]:
model_b1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 20
Epoch 1/15
34/34 [==============================] - ETA: 0s - loss: 2.5895 - categorical_accuracy: 0.3092Source path =  /home/datasets/Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2022-05-1805_39_49.403993/model-00001-2.58950-0.30920-2.14031-0.21000.h5
34/34 [==============================] - 189s 6s/step - loss: 2.5895 - categorical_accuracy: 0.3092 - val_loss: 2.1403 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/15
34/34 [==============================] - ETA: 0s - loss: 2.4839 - categorical_accuracy: 0.2843
Epoch 00002: saving model to model_init_2022-05-1805_39_49.403993/model-00002-2.48388-0.28431-2.68283-0.21000.h5
34/34 [==============================] - 63s 2s/step - loss: 2.4839 - categorical_accuracy: 0.2843 - val_loss: 2.6828 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/15
34/34 [==============================] - ETA: 0s - loss: 2.6395 - categorical_accuracy

From exp-2 and exp-3 there is very small change in training and validation accuracy even though we increase the no of layers and no of epochs.We can see the model is more impacted by image resolution,batch size and no of frames.

#### Exp 4:change the image resolution to 80x80  ,with frames=30 and  by keeping batchsize=10 and epochs=20

In [21]:
x = 30 # number of frames
y = 80 # image width
z = 80 # image height

batch_size=10
num_epochs=20

In [22]:
model_b2 = Sequential()
model_b2.add(Conv3D(8, kernel_size=(3, 3, 3), input_shape=(x,y,z,channels), padding='same'))
model_b2.add(Activation('relu'))
model_b2.add(BatchNormalization())



model_b2.add(MaxPooling3D(pool_size=(2, 2, 2)))


model_b2.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model_b2.add(Activation('relu'))
model_b2.add(BatchNormalization())

model_b2.add(MaxPooling3D(pool_size=(2, 2, 2)))


model_b2.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_b2.add(Activation('relu'))
model_b2.add(BatchNormalization())

model_b2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_b2.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
model_b2.add(Activation('relu'))
model_b2.add(BatchNormalization())

model_b2.add(MaxPooling3D(pool_size=(2, 2, 2)))


model_b2.add(Flatten())
model_b2.add(Dense(1000, activation='relu'))
model_b2.add(Dropout(0.5))
model_b2.add(Dense(500, activation='relu'))
model_b2.add(Dropout(0.5))
model_b2.add(Dense(classes, activation='softmax'))

In [23]:
optimiser = 'adam'
model_b2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_b2.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 80, 80, 8)     656       
                                                                 
 activation_4 (Activation)   (None, 30, 80, 80, 8)     0         
                                                                 
 batch_normalization_4 (Batc  (None, 30, 80, 80, 8)    32        
 hNormalization)                                                 
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 15, 40, 40, 8)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 40, 40, 16)    3472      
                                                                 
 activation_5 (Activation)   (None, 15, 40, 40, 16)   

In [24]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [32]:
model_b2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 3.2698 - categorical_accuracy: 0.2941Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1805_39_49.403993/model-00001-3.26976-0.29412-4.54848-0.16000.h5
67/67 [==============================] - 189s 3s/step - loss: 3.2698 - categorical_accuracy: 0.2941 - val_loss: 4.5485 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 2.1113 - categorical_accuracy: 0.3483
Epoch 00002: saving model to model_init_2022-05-1805_39_49.403993/model-00002-2.11131-0.34826-5.95769-0.16000.h5
67/67 [==============================] - 76s 1s/step - loss: 2.1113 - categorical_accuracy: 0.3483 - val_loss: 5.9577 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 2.1355 - categorical_accuracy

Here training accuracy of 79% and validation accuracy of 73%. So this model is a good model and can be used for further analysis.



####  For now CONV 3D model the model in exp 4 is the best one.

## Conv2D + RNN to build a model

In [13]:
x = 30 # number of frames
y = 100 # image width
z = 100 # image height
batch_size=10
num_epochs=30
channels=3
classes=5


In [14]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[x for x in range(0,x)]   #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size    # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [15]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =   30  # choose the number of epochs
print ('# epochs =', num_epochs)
num_batches = num_train_sequences//batch_size 
print(num_batches)

# training sequences = 663
# validation sequences = 100
# epochs = 30
66


#### Exp- 5 :custom conv2d + GRU 

In [16]:
#define model

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.layers import Dropout


input_shape=(x,y,z,channels)
model_d= Sequential()

model_d.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3),  padding='same'),input_shape=input_shape))
model_d.add(TimeDistributed(Activation('relu')))
model_d.add(TimeDistributed(BatchNormalization()))


model_d.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same')))
model_d.add(TimeDistributed(Activation('relu')))
model_d.add(TimeDistributed(BatchNormalization()))


model_d.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3),  padding='same')))
model_d.add(TimeDistributed(Activation('relu')))
model_d.add(TimeDistributed(BatchNormalization()))


model_d.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), padding='same')))
model_d.add(TimeDistributed(Activation('relu')))
model_d.add(TimeDistributed(BatchNormalization()))


model_d.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d.add(TimeDistributed(Flatten()))

model_d.add(GRU(64))
model_d.add(Dropout(0.25))

model_d.add(Dense(64,activation='relu'))
model_d.add(Dropout(0.25))

model_d.add(Dense(classes, activation='softmax'))





In [17]:
optimiser=tf.keras.optimizers.Adam()
model_d.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_d.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 100, 100, 16)  448      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 100, 100, 16)  0        
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 100, 100, 16)  64       
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 50, 50, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 50, 50, 32)  

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, cooldown=1, verbose=1)# write the REducelronplateau code here
earlystop = EarlyStopping( monitor="val_loss", min_delta=0,patience=10,verbose=1)
callbacks_list = [checkpoint, LR, earlystop]

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [41]:
model_d.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 1.5597 - categorical_accuracy: 0.3243Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1806_37_32.418090/model-00001-1.55970-0.32428-1.69218-0.22000.h5
67/67 [==============================] - 184s 3s/step - loss: 1.5597 - categorical_accuracy: 0.3243 - val_loss: 1.6922 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.4174 - categorical_accuracy: 0.3980
Epoch 00002: saving model to model_init_2022-05-1806_37_32.418090/model-00002-1.41735-0.39801-1.60546-0.24000.h5
67/67 [==============================] - 73s 1s/step - loss: 1.4174 - categorical_accuracy: 0.3980 - val_loss: 1.6055 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.4420 - categorical_accuracy

from the above model we can infer that the accuracy of training and validation are 56% and 55.0% respectively.<br>

#### Exp-6 : Let us add more dense nuerons and gru cells and examine the results 

In [42]:
input_shape=(x,y,z,channels)
model_d1= Sequential()

model_d1.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3),  padding='same'),input_shape=input_shape))
model_d1.add(TimeDistributed(Activation('relu')))
model_d1.add(TimeDistributed(BatchNormalization()))


model_d1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d1.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same')))
model_d1.add(TimeDistributed(Activation('relu')))
model_d1.add(TimeDistributed(BatchNormalization()))


model_d1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d1.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3),  padding='same')))
model_d1.add(TimeDistributed(Activation('relu')))
model_d1.add(TimeDistributed(BatchNormalization()))


model_d1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d1.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), padding='same')))
model_d1.add(TimeDistributed(Activation('relu')))
model_d1.add(TimeDistributed(BatchNormalization()))


model_d1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d1.add(TimeDistributed(Flatten()))

model_d1.add(GRU(128))
model_d1.add(Dropout(0.25))

model_d1.add(Dense(128,activation='relu'))
model_d1.add(Dropout(0.25))

model_d1.add(Dense(classes, activation='softmax'))



In [43]:
optimiser=tf.keras.optimizers.Adam()
model_d1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_d1.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_17 (TimeDi  (None, 30, 100, 100, 16)  448      
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 30, 100, 100, 16)  0        
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 30, 100, 100, 16)  64       
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 30, 50, 50, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 30, 50, 50, 32)  

In [44]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [45]:
model_d1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 1.5250 - categorical_accuracy: 0.3635Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1806_37_32.418090/model-00001-1.52503-0.36350-2.23002-0.16000.h5
67/67 [==============================] - 186s 3s/step - loss: 1.5250 - categorical_accuracy: 0.3635 - val_loss: 2.2300 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.5209 - categorical_accuracy: 0.3483
Epoch 00002: saving model to model_init_2022-05-1806_37_32.418090/model-00002-1.52086-0.34826-2.39731-0.19000.h5
67/67 [==============================] - 75s 1s/step - loss: 1.5209 - categorical_accuracy: 0.3483 - val_loss: 2.3973 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.4937 - categorical_accuracy

#### Improvement in accuracy is observed

#### Exp-7: add layers in GRU 

In [32]:
input_shape=(x,y,z,channels)
model_d2= Sequential()

model_d2.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3),  padding='same'),input_shape=input_shape))
model_d2.add(TimeDistributed(Activation('relu')))
model_d2.add(TimeDistributed(BatchNormalization()))


model_d2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d2.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same')))
model_d2.add(TimeDistributed(Activation('relu')))
model_d2.add(TimeDistributed(BatchNormalization()))


model_d2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d2.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3),  padding='same')))
model_d2.add(TimeDistributed(Activation('relu')))
model_d2.add(TimeDistributed(BatchNormalization()))


model_d2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d2.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), padding='same')))
model_d2.add(TimeDistributed(Activation('relu')))
model_d2.add(TimeDistributed(BatchNormalization()))


model_d2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d2.add(TimeDistributed(Flatten()))

model_d2.add(GRU(128,return_sequences=True))
model_d2.add(Dropout(0.25))

model_d2.add(GRU(128))
model_d2.add(Dropout(0.25))


model_d2.add(Dense(128,activation='relu'))
model_d2.add(Dropout(0.25))

model_d2.add(Dense(classes, activation='softmax'))



In [33]:
optimiser=tf.keras.optimizers.Adam()
model_d2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_d2.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_17 (TimeDi  (None, 30, 80, 80, 16)   448       
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 30, 80, 80, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 30, 80, 80, 16)   64        
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 30, 40, 40, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 30, 40, 40, 32)  

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model_d2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/20


2022-05-18 12:06:58.250310: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


67/67 [==============================] - ETA: 0s - loss: 1.4810 - categorical_accuracy: 0.3650Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1812_05_17.124508/model-00001-1.48097-0.36501-2.01979-0.16000.h5
67/67 [==============================] - 171s 3s/step - loss: 1.4810 - categorical_accuracy: 0.3650 - val_loss: 2.0198 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.4985 - categorical_accuracy: 0.4030
Epoch 00002: saving model to model_init_2022-05-1812_05_17.124508/model-00002-1.49851-0.40299-2.38310-0.15000.h5
67/67 [==============================] - 70s 1s/step - loss: 1.4985 - categorical_accuracy: 0.4030 - val_loss: 2.3831 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.4962 - categorical_accuracy: 0.2985
Epoch 00003: saving model to model_init_2022-05-1812_05_17.124508/mod

Training accuracy is 79% and validation accuracy is 69%. Let try to improve it further as there is a slight possibility of overfitting.

#### Exp-8 :Add dropouts,and set the learning rate to 0.0001

In [21]:
input_shape=(x,y,z,channels)
model_d3= Sequential()

model_d3.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3),  padding='same'),input_shape=input_shape))
model_d3.add(TimeDistributed(Activation('relu')))
model_d3.add(TimeDistributed(BatchNormalization()))


model_d3.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))


model_d3.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same')))
model_d3.add(TimeDistributed(Activation('relu')))
model_d3.add(TimeDistributed(BatchNormalization()))


model_d3.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_d3.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3),  padding='same')))
model_d3.add(TimeDistributed(Activation('relu')))
model_d3.add(TimeDistributed(BatchNormalization()))


model_d3.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))


model_d3.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), padding='same')))
model_d3.add(TimeDistributed(Activation('relu')))
model_d3.add(TimeDistributed(BatchNormalization()))


model_d3.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model_d3.add(Dropout(0.25))

model_d3.add(TimeDistributed(Flatten()))

model_d3.add(GRU(128,return_sequences=True))
model_d3.add(Dropout(0.25))

model_d3.add(GRU(128))
model_d3.add(Dropout(0.25))


model_d3.add(Dense(128,activation='relu'))
model_d3.add(Dropout(0.25))

model_d3.add(Dense(classes, activation='softmax'))



In [22]:
optimiser=tf.keras.optimizers.Adam(0.0001)
model_d3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_d3.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_17 (TimeDi  (None, 30, 100, 100, 16)  448      
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 30, 100, 100, 16)  0        
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 30, 100, 100, 16)  64       
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 30, 50, 50, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 30, 50, 50, 32)  

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_d3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/30


2022-05-18 12:47:15.380647: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


67/67 [==============================] - ETA: 0s - loss: 1.5062 - categorical_accuracy: 0.3213Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1812_46_48.418371/model-00001-1.50616-0.32127-1.70813-0.16000.h5
67/67 [==============================] - 167s 2s/step - loss: 1.5062 - categorical_accuracy: 0.3213 - val_loss: 1.7081 - val_categorical_accuracy: 0.1600 - lr: 1.0000e-04
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.2608 - categorical_accuracy: 0.4726
Epoch 00002: saving model to model_init_2022-05-1812_46_48.418371/model-00002-1.26085-0.47264-1.83875-0.15000.h5
67/67 [==============================] - 67s 1s/step - loss: 1.2608 - categorical_accuracy: 0.4726 - val_loss: 1.8387 - val_categorical_accuracy: 0.1500 - lr: 1.0000e-04
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.2398 - categorical_accuracy: 0.5174
Epoch 00003: saving model to model_init_2022-05-1812_46_48.41

 training accuracy is 80% and the validation accuracy is 79.0%. But the loss is too high and it didnt decreased.

## Thus final model is Exp-8 with Training Accuracy: 0.80, Validation accuracy: 0.79 with least number of parameters.